# Import Library

In [1]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img, image
from keras.utils import to_categorical
from keras.models import Sequential
import os



Using TensorFlow backend.


# Define Constants

In [2]:
FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

# Define Model Architecture

In [4]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

def createModel() -> Sequential:
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [5]:
from keras.models import load_model

model = createModel()
model.load_weights('model.h5')

In [6]:
def load(filename: str):
    img = image.load_img(filename, target_size = IMAGE_SIZE)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    return img

In [7]:
img = load('dog.jpg')

result = model.predict(img)

if result[0][0] == 1:
    print('That is a cat')
else:
    print('That is a dog')


That is a dog


# Convert now to an ML model

In [11]:
import coremltools

new_model = coremltools.converters.keras.convert(model,
    input_names="image",
    image_input_names="image",
    image_scale=1/255.0)

0 : conv2d_1_input, <keras.engine.input_layer.InputLayer object at 0x12fd27d68>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x110717b00>
2 : conv2d_1__activation__, <keras.layers.core.Activation object at 0x138c306d8>
3 : batch_normalization_1, <keras.layers.normalization.BatchNormalization object at 0x12fd27e48>
4 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x103ee70b8>
5 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x12fd97400>
6 : conv2d_2__activation__, <keras.layers.core.Activation object at 0x138e80320>
7 : batch_normalization_2, <keras.layers.normalization.BatchNormalization object at 0x12feb54a8>
8 : max_pooling2d_2, <keras.layers.pooling.MaxPooling2D object at 0x12ff32668>
9 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x12ffbb438>
10 : conv2d_3__activation__, <keras.layers.core.Activation object at 0x138e806a0>
11 : batch_normalization_3, <keras.layers.normalization.BatchNormalization object at 0x12ffdb908>
12 : max_p

# Save new ML model

In [12]:
new_model.save('cat_dog.mlmodel')